In [1]:
# 1. Imports & seed
import numpy as np
import pandas as pd
import os, random
from sklearn.metrics import cohen_kappa_score

def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(42)

def qwk(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


In [4]:
# 2. Load labels only (ID was dropped upstream, this is expected)

train_df = pd.read_csv("train_final.csv")

# Extract true labels
y_true = train_df["sii"].values
print("Train label shape:", y_true.shape)

# For test IDs, always use sample_submission.csv
sub = pd.read_csv("sample_submission.csv")
test_ids = sub["id"].values
print("Test id shape:", test_ids.shape)


Train label shape: (2736,)
Test id shape: (20,)


In [14]:
# 3. Load Level-1 OOF & test predictions from 7_Advanced_Techniques
oof_lgbm = pd.read_csv("oof_final_lgbm.csv")
oof_xgb  = pd.read_csv("oof_final_xgb.csv")
oof_cat  = pd.read_csv("oof_final_cat.csv")

test_lgbm = pd.read_csv("test_final_lgbm.csv")
test_xgb  = pd.read_csv("test_final_xgb.csv")
test_cat  = pd.read_csv("test_final_cat.csv")

oof_list  = [oof_lgbm, oof_xgb, oof_cat]
test_list = [test_lgbm, test_xgb, test_cat]
names     = ["LGBM", "XGB", "CatBoost"]

oof_list  = [np.asarray(oof) for oof in oof_list]
test_list = [np.asarray(test) for test in test_list]

for name, oof in zip(names, oof_list):
    preds = oof.argmax(axis=1)
    print(f"{name} OOF QWK: {qwk(y_true, preds):.4f}")


LGBM OOF QWK: 0.4508
XGB OOF QWK: 0.4461
CatBoost OOF QWK: 0.4200


In [15]:
# 4. Simple weighted blending (random Dirichlet search)
def eval_weights(w, oofs, y):
    blend = np.zeros_like(oofs[0])
    for wi, p in zip(w, oofs): blend += wi * p
    return qwk(y, blend.argmax(1)), blend

def search_blend(oofs, y, n_trials=500, seed=42):
    rng = np.random.RandomState(seed)
    m = len(oofs); best_q = -1; best_w = None; best_blend = None
    for t in range(n_trials):
        w = rng.dirichlet(np.ones(m))
        q, b = eval_weights(w, oofs, y)
        if q > best_q: best_q, best_w, best_blend = q, w, b
    return best_q, best_w, best_blend

best_qwk_blend, best_w, oof_blend = search_blend(oof_list, y_true, n_trials=500)
print("Best blend QWK:", round(best_qwk_blend, 4))
print("Best weights:", dict(zip(names, best_w)))


Best blend QWK: 0.451
Best weights: {'LGBM': np.float64(0.5935798121859749), 'XGB': np.float64(0.40161224689521285), 'CatBoost': np.float64(0.0048079409188122755)}


In [17]:
# 5. Apply same weights to test predictions
test_blend = np.zeros_like(test_list[0])
for wi, p in zip(best_w, test_list): test_blend += wi * p
print("Test blend shape:", test_blend.shape)


Test blend shape: (20, 4)


In [18]:
# 6. Hill-climbing post-processing (copied from 7_Advanced_Techniques)
def hill_climb_qwk(y_true, proba, n_iter=3000, seed=42):
    rng = np.random.RandomState(seed)
    n, k = proba.shape
    preds = proba.argmax(1)
    best_preds = preds.copy()
    best_q = qwk(y_true, best_preds)
    for it in range(n_iter):
        idx = rng.randint(0, n)
        new_c = rng.randint(0, k)
        old_c = best_preds[idx]
        if new_c == old_c: continue
        best_preds[idx] = new_c
        q_new = qwk(y_true, best_preds)
        if q_new > best_q: best_q = q_new
        else: best_preds[idx] = old_c
    return best_preds, best_q

# Hill-climb on blended OOF
hc_oof_labels, hc_oof_qwk = hill_climb_qwk(y_true, oof_blend, n_iter=3000)
print("Hill-climbed OOF QWK:", round(hc_oof_qwk, 4))


Hill-climbed OOF QWK: 0.547


In [19]:
# 7. Map hill-climb to test: simple approach = keep test_blend argmax (no labels to optimize on)
final_test_preds = test_blend.argmax(1)
print("Pred label distribution (test):", np.unique(final_test_preds, return_counts=True))


Pred label distribution (test): (array([0, 1, 2]), array([12,  4,  4]))


In [20]:
# 8. Integrity checks: permutation test on blended OOF
def permutation_test(proba, y, n_perm=3, seed=42):
    rng = np.random.RandomState(seed)
    scores = []
    for i in range(n_perm):
        yp = rng.permutation(y)
        scores.append(qwk(yp, proba.argmax(1)))
    return np.array(scores)

perm_scores = permutation_test(oof_blend, y_true, n_perm=3)
print("True blended QWK:", round(best_qwk_blend, 4))
print("Permuted QWKs:", perm_scores.round(4))


True blended QWK: 0.451
Permuted QWKs: [-0.0049  0.0099  0.02  ]


In [23]:
# 9. Final submission
submission = pd.read_csv("sample_submission.csv")
submission["sii"] = final_test_preds.astype(int)
assert submission["sii"].between(0, 20).all()
submission.to_csv("submission.csv", index=False)
submission.head()


,id,sii
0,00008ff9,2
1,000fd460,0
2,00105258,0
3,00115b9f,1
4,0016bb22,0
